In [6]:
from langchain_community.document_loaders import PyPDFLoader
loader=PyPDFLoader("../data/Smart_Job_Internship_Recommendation_Platform__1768917114.pdf")
     


In [7]:
from pathlib import Path
import os
def documents_procesors(dir_name):
    documents_list=[]
    dir_name_p=Path(dir_name)

    file_names=dir_name_p.glob("**/*.pdf")

    for file_name in file_names:
        loader=PyPDFLoader(file_name)
        documents=loader.load()
        for doc in documents:
            doc.metadata["file_name"]=file_name.name
            doc.metadata["file_type"]="pdf"

        documents_list.extend(documents)
        print(documents)
    return documents_list
      
all_pdf_documents=documents_procesors("../data/")

[Document(metadata={'producer': 'pdfTeX-1.40.27', 'creator': 'LaTeX with hyperref', 'creationdate': '2026-01-15T13:16:22+00:00', 'author': '', 'keywords': '', 'moddate': '2026-01-15T13:16:22+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.27 (TeX Live 2025) kpathsea version 6.4.1', 'subject': '', 'title': '', 'trapped': '/False', 'source': '..\\data\\Smart_Job_Internship_Recommendation_Platform__1768917114.pdf', 'total_pages': 45, 'page': 0, 'page_label': '1', 'file_name': 'Smart_Job_Internship_Recommendation_Platform__1768917114.pdf', 'file_type': 'pdf'}, page_content='Ibn Tofail University - ENSAK\nProgram: Computer Engineering\nProject Report\nJava Application for Internship and Job Search in\nMorocco\nPrepared by:\nRhemimet Meryem\nKhaliqi Salma\nSupervised by:\nProf. Dr. Moumen Aniss\nAcademic Year: 2024 – 2025'), Document(metadata={'producer': 'pdfTeX-1.40.27', 'creator': 'LaTeX with hyperref', 'creationdate': '2026-01-15T13:16:22+00:00', 'author': '', 'k

In [8]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

def text_splitter(documents,chunk_size=1000,chunk_overlap=200):
    Text_splitter=RecursiveCharacterTextSplitter(chunk_size= chunk_size,chunk_overlap= chunk_overlap)
    chunks = Text_splitter.split_documents(documents)

    return chunks


chunks= text_splitter(all_pdf_documents)

In [9]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS


class EmbeddingsManager:

   def __init__(self,model_name: str="sentence-transformers/all-MiniLM-L6-v2"):
        self.model_name=model_name
        self.embeddings=None
        self._load_modle()
   
   def _load_modle(self):
     print(f"Loading embedding model: {self.model_name}")
     self.embeddings= HuggingFaceEmbeddings(model_name=self.model_name)
   def get_embeddings(self):
     return self.embeddings


embeddingmanager = EmbeddingsManager()
embeddings = embeddingmanager.get_embeddings()


vectorstore=FAISS.from_documents(chunks,embeddings)
vectorstore.save_local("faiss_index")

Loading embedding model: sentence-transformers/all-MiniLM-L6-v2


Loading weights: 100%|██████████| 103/103 [00:00<00:00, 166.76it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
